In [162]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directorye

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [163]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [164]:
train.dtypes

In [165]:
train.dtypes.value_counts().sum()

In [166]:
test.dtypes.value_counts().sum()

In [167]:
train.head()

In [168]:
test.head()

In [169]:
ytrain = train['SalePrice']

In [170]:
import matplotlib.pyplot as plt
import seaborn as sns

In [171]:
fig = plt.figure(figsize = (10,10) )
ax = fig.subplots(1,1)
ax.scatter(train['GarageArea'],ytrain)
ax.set_xlabel('GarageArea')
ax.set_ylabel('SalePrice')
plt.show()

In [172]:
corr = train.corr()
corr
fx,ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr)

In [173]:
Sale_relation = pd.DataFrame(corr['SalePrice'])
Sale_relation = Sale_relation.sort_values(by = 'SalePrice',ascending = False)
Sale_relation

In [174]:
top10 = Sale_relation.index[:11]

In [175]:
top10

In [176]:
top10data = train[top10]
top10corr = top10data.corr()
f,ax = plt.subplots(figsize = (10,10)) 
#sns.set(font_scale=1.25) 
sns.heatmap(top10corr,  annot=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=top10data.columns, xticklabels=top10data.columns)

In [177]:
sns.pairplot(top10data[top10[:5]],size=2.5)

In [178]:
top10test = test[top10[1:]]

In [179]:
train.isnull().sum().sort_values(ascending=False)

In [180]:
from sklearn.preprocessing import StandardScaler
scaled = top10data[:]
scaled['SalePrice'] = StandardScaler().fit_transform(scaled['SalePrice'][:,np.newaxis])

In [181]:
top10data.drop(['1stFlrSF','FullBath','TotRmsAbvGrd','YearRemodAdd','GarageArea'],1,inplace=True)
top10test.drop(['1stFlrSF','FullBath','TotRmsAbvGrd','YearRemodAdd','GarageArea'],1,inplace=True)

In [182]:
from scipy.stats import norm
sns.distplot((top10data['SalePrice']),fit=norm)

In [183]:
top10data['SalePrice']=np.log(top10data['SalePrice'])
ytrain = top10data['SalePrice']
top10data.drop('SalePrice',1,inplace = True)

In [184]:
top10test['Id'] = test['Id']
top10data['Id'] = train['Id']

In [185]:
sns.distplot(ytrain,fit=norm)

In [186]:
sns.distplot(top10data['GrLivArea'],fit=norm)

In [187]:
top10data['GrLivArea'] = np.log(top10data['GrLivArea'])
top10test['GrLivArea'] = np.log(top10data['GrLivArea'])

In [188]:
sns.distplot(top10data['GrLivArea'],fit=norm)

In [189]:
sns.distplot(top10data['TotalBsmtSF'], fit=norm);

In [190]:
top10data['hasbsmt'] = 0
top10test['hasbsmt'] = 0
top10data.head()

In [191]:
top10data.loc[top10data['TotalBsmtSF']>0.0,'hasbsmt'] = 1 
top10test.loc[top10test['TotalBsmtSF']>0.0,'hasbsmt'] = 1 

In [192]:
top10test.head()

In [193]:
top10data.loc[top10data['hasbsmt']==1,'TotalBsmtSF']=np.log(top10data['TotalBsmtSF'])
top10test.loc[top10test['hasbsmt']==1,'TotalBsmtSF']=np.log(top10test['TotalBsmtSF'])

In [194]:
sns.distplot(top10data[top10data['TotalBsmtSF']>0]['TotalBsmtSF'],fit=norm)

In [195]:
top10test.head()

In [196]:
full = pd.concat([top10data,top10test])

In [197]:
catfull = pd.concat([full,pd.get_dummies(full['OverallQual'])],1)

In [198]:
catfull = pd.concat([catfull,pd.get_dummies(full['GarageCars'])],1)

In [199]:
catfull.drop(['GarageCars','OverallQual'],inplace=True,axis=1)

In [200]:
cat_train = catfull[:1460]

In [201]:
catfull.head()

In [202]:
cat_test = catfull[1460:]

In [203]:
train.shape

In [204]:
cat_test.head()

In [205]:
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.linear_model import LinearRegression as lr
from sklearn.tree import DecisionTreeRegressor as dt
from sklearn import svm 
from sklearn.model_selection import train_test_split

In [206]:
ytrain.shape

In [207]:
x_train,x_cross,y_train,y_cross = train_test_split(cat_train,ytrain,test_size=0.33,random_state=42)

In [208]:
x_train.head()

In [209]:
model = dt()
model.fit(x_train,pd.DataFrame(y_train))

In [210]:
y_obt = model.predict(x_cross)

In [211]:
from sklearn.metrics import average_precision_score as pscr
from sklearn.metrics import r2_score as ascr
ascr(y_cross,y_obt)

In [212]:
xtrainid = cat_train['Id']
xtestid = cat_test['Id']

In [213]:
model.fit(cat_train,ytrain)

In [214]:
cat_test[cat_test['TotalBsmtSF'].isnull()]

In [215]:
cat_test['TotalBsmtSF'] = cat_test['TotalBsmtSF'].fillna(6.714643)

In [216]:
model.fit(cat_train,ytrain)

In [217]:
ypredtest = model.predict(cat_test)

In [218]:
ypredtest

In [219]:
lamb = lambda x: np.power(np.e,x)
ypredtest = np.array([lamb(i) for i in ypredtest])

In [220]:
cat_test.head(10)

In [221]:
submission = pd.DataFrame({'Id':xtestid, 'SalePrice':ypredtest})

In [222]:
 submission

In [223]:
submission.to_csv('submission1.csv',index=False)